In [ ]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.0.0'
os.environ['SPARK_VERSION']=spark_version

In [ ]:
# innstall java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 114 

In [ ]:
# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-02-19 23:32:57--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.5’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.70MB/s    in 0.2s    

2023-02-19 23:32:57 (5.70 MB/s) - ‘postgresql-42.2.9.jar.5’ saved [914037/914037]



In [ ]:
pip uninstall pyspark

Found existing installation: pyspark 3.0.0
Uninstalling pyspark-3.0.0:
  Would remove:
    /usr/local/bin/beeline
    /usr/local/bin/beeline.cmd
    /usr/local/bin/docker-image-tool.sh
    /usr/local/bin/find-spark-home
    /usr/local/bin/find-spark-home.cmd
    /usr/local/bin/find_spark_home.py
    /usr/local/bin/load-spark-env.cmd
    /usr/local/bin/load-spark-env.sh
    /usr/local/bin/pyspark
    /usr/local/bin/pyspark.cmd
    /usr/local/bin/pyspark2.cmd
    /usr/local/bin/run-example
    /usr/local/bin/run-example.cmd
    /usr/local/bin/spark-class
    /usr/local/bin/spark-class.cmd
    /usr/local/bin/spark-class2.cmd
    /usr/local/bin/spark-shell
    /usr/local/bin/spark-shell.cmd
    /usr/local/bin/spark-shell2.cmd
    /usr/local/bin/spark-sql
    /usr/local/bin/spark-sql.cmd
    /usr/local/bin/spark-sql2.cmd
    /usr/local/bin/spark-submit
    /usr/local/bin/spark-submit.cmd
    /usr/local/bin/spark-submit2.cmd
    /usr/local/bin/sparkR
    /usr/local/bin/sparkR.cmd
    /usr/lo

In [ ]:
pip install pyspark==3.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pyspark-3.0.0-py2.py3-none-any.whl


In [ ]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [ ]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Video_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   49033728|R1P1G5KZ05H6RD|6302503213|     748506413|The Night They Sa...|           Video|          5|            0|          0|   N|                Y|    Very satisfied!!|Fast shipping. Pl...|2015-01-31 00:08:00|
|         US|   17857748|R106N066IUN8ZV|B000059PET|     478710180|Hamlet / Kline, N...| 

In [ ]:
# Get the number of rows in the DataFrame.
df.count()

380604

In [ ]:
df = df.dropna()

print(df.count())

380575


# Transform the Data

In [ ]:
# Print schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



## Create the "review_id_table".

In [ ]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = df.select(["review_id","customer_id","product_id", "product_parent","review_date"])
review_id_df.show(20)

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R1P1G5KZ05H6RD|   49033728|6302503213|     748506413|2015-01-31 00:08:00|
|R106N066IUN8ZV|   17857748|B000059PET|     478710180|2015-01-31 00:08:00|
| R7WTAA1S5O7D9|   25551507|0788812807|     981002815|2015-01-31 00:08:00|
|R32HFMVWLYOYJK|   21025041|6302509939|     333219811|2015-01-31 00:08:00|
| RWT3H6HBVAL6G|   40943563|B00JENS2BI|     538101194|2015-01-31 00:08:00|
|R1S3T3GWUGQTW7|   17013969|6305761302|     716303278|2015-01-31 00:08:00|
|R3R0QYHA800REE|   47611685|6300157555|     134996462|2015-01-31 00:08:00|
|R1FR0EQPHPW5NM|   35680737|6300189570|     498116870|2015-01-31 00:08:00|
| RGORN81H45NI7|   10747909|B000SXQ5US|      77519275|2015-01-31 00:08:00|
|R1CNYN4ABNOJSU|     126341|B00008F22G|     917778300|2015-01-31 00:08:00|
|R2DW06821PMN40|   406768

## Create the "products" Table

In [ ]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = df.select(["product_id","product_title"])
products_df = products_df.dropDuplicates()
products_df.show(20)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0788806270|Bambi (Walt Disne...|
|078881107X|Kiki's Delivery S...|
|6303559018|To Catch a Yeti [...|
|6302844061|      Silkwood [VHS]|
|B00003XAMY|Wagons Roll at Ni...|
|0783216084|          Jaws [VHS]|
|6301966376|The Compleat Beat...|
|6303315429| Yoga for Life [VHS]|
|6302969204|Miracle in Milan ...|
|6301540441|As Summers Die [VHS]|
|6304611366|Loretta Young Sho...|
|1559838450|Hans Christian An...|
|6304263198|   Latcho Drom [VHS]|
|6301017250|Dead Don't Die [VHS]|
|6301782135|John Hammond: Fro...|
|6304925158|The Education of ...|
|6305403309| Dead Husbands [VHS]|
|6300152936|       Faeries [VHS]|
|B0000897E9|  Femme Fatale [VHS]|
|B0002X4MIQ|Touched By An Ang...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [ ]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
from pyspark.sql.functions import desc

customers_df = df.groupby("customer_id").agg({"customer_id":"count"})
customers_df = customers_df.orderBy(desc("count(customer_id)"))
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count") 
customers_df.show(20)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   50913245|          1005|
|   38002140|           953|
|   18116317|           898|
|   49778371|           765|
|   50763106|           648|
|   50205849|           580|
|   49355567|           567|
|   26803373|           559|
|   50736950|           534|
|    7080939|           506|
|   49837360|           422|
|   39371720|           394|
|   50303974|           391|
|   50237277|           387|
|   51299750|           382|
|   53016962|           380|
|   51010646|           334|
|   49786731|           328|
|   49818928|           315|
|   43921586|           310|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [ ]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
# Load in a sql function to use columns
from pyspark.sql.functions import col

vine_table_df = df.select(["review_id","star_rating","helpful_votes", "total_votes","vine"])

# # Filter for only columns from Amazon's Vine program
# vine_table_df = vine_table_df.filter(col("vine")  == "Y")
vine_table_df.show(20) 

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1P1G5KZ05H6RD|          5|            0|          0|   N|
|R106N066IUN8ZV|          5|            0|          0|   N|
| R7WTAA1S5O7D9|          4|            0|          0|   N|
|R32HFMVWLYOYJK|          5|            0|          0|   N|
| RWT3H6HBVAL6G|          3|            0|          0|   N|
|R1S3T3GWUGQTW7|          5|            0|          0|   N|
|R3R0QYHA800REE|          4|            0|          0|   N|
|R1FR0EQPHPW5NM|          4|            1|          2|   N|
| RGORN81H45NI7|          5|            1|          1|   N|
|R1CNYN4ABNOJSU|          5|            0|          0|   N|
|R2DW06821PMN40|          3|            0|          0|   N|
|R1CS8AMA8B0VBJ|          5|            0|          0|   N|
|R343CPRI4MC9J0|          5|            0|          0|   N|
| R3XP0G8P2BOTP|          5|            

# Load

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://<>:5432/my_data_class_db"
config = {"user":"postgres", "password": "", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)